In [19]:
import os 
import json
import pandas as pd
import traceback

In [20]:
from langchain.chat_models import ChatOpenAI

In [21]:
from dotenv import load_dotenv
load_dotenv() #take environment variables from .env 

True

In [22]:
# First writedown your own OPENAI API KEY
KEY = os.getenv('OPENAI_API_KEY')

In [23]:
llm = ChatOpenAI(openai_api_key=KEY, model_name='gpt-3.5-turbo', temperature='0.5')

In [24]:
from langchain .llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2

In [25]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [26]:
TEMPLATE="""
Text{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz of {number} multiple choice questions for {subject} students in {tone} tone.
Make sute the questions are not repeated and check all the questions to be confirmed the text as well.
Make sure to format your response like RESPONSE_JSON below and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

"""

In [27]:
quize_generation_prompt = PromptTemplate(
    input_variables=['text','number','subject','tone','response_json'],
    template=TEMPLATE
)

In [28]:
quiz_chain = LLMChain(llm=llm, prompt=quize_generation_prompt, output_key='quiz', verbose=True)

In [29]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [30]:
quize_evaluation_prompt = PromptTemplate(
    input_variables=['subject','quiz'], 
    template=TEMPLATE2
)

In [31]:
review_chain = LLMChain(llm=llm, prompt=quize_evaluation_prompt, output_key='review', verbose=True)

In [32]:
generate_evaluate_chain = SequentialChain(
    chains=[quiz_chain, review_chain], 
    input_variables=['text','number','subject','tone','response_json'],
    output_variables=['quiz','review'],
    verbose=True
)

In [36]:
#reading the data.txt 
file_path = r'C:\Users\UpforceTech\mcqgen\data.txt'

with open(file_path,'r') as file:
    TEXT = file.read()

print(TEXT)

Artificial intelligence

Machine learning as subfield of AI[22]
As a scientific endeavor, machine learning grew out of the quest for artificial intelligence (AI). In the early days of AI as an academic discipline, some researchers were interested in having machines learn from data. They attempted to approach the problem with various symbolic methods, as well as what were then termed "neural networks"; these were mostly perceptrons and other models that were later found to be reinventions of the generalized linear models of statistics.[23] Probabilistic reasoning was also employed, especially in automated medical diagnosis.[24]: 488 

However, an increasing emphasis on the logical, knowledge-based approach caused a rift between AI and machine learning. Probabilistic systems were plagued by theoretical and practical problems of data acquisition and representation.[24]: 488  By 1980, expert systems had come to dominate AI, and statistics was out of favor.[25] Work on symbolic/knowledge-ba

In [37]:
# Conveting the python dictionary into a JSON-formatted string
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [43]:
NUMBER = 5
SUBJECT ='Machine Learning'
TONE = 'simple'

In [44]:
# To set up token usage tracking in langchain
with get_openai_callback() as cb:
    response = generate_evaluate_chain(
        {
            'text':TEXT,
            'number':NUMBER,
            'subject':SUBJECT,
            'tone':TONE,
            'response_json': json.dumps(RESPONSE_JSON)
        }
    )



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

TextArtificial intelligence

Machine learning as subfield of AI[22]
As a scientific endeavor, machine learning grew out of the quest for artificial intelligence (AI). In the early days of AI as an academic discipline, some researchers were interested in having machines learn from data. They attempted to approach the problem with various symbolic methods, as well as what were then termed "neural networks"; these were mostly perceptrons and other models that were later found to be reinventions of the generalized linear models of statistics.[23] Probabilistic reasoning was also employed, especially in automated medical diagnosis.[24]: 488 

However, an increasing emphasis on the logical, knowledge-based approach caused a rift between AI and machine learning. Probabilistic systems were plagued by theoretical and practical problems of data acquisition and representation.[24]: 488  By 1980,

In [45]:
print(f"Total Tokens:{cb.total_tokens}")
print(f"Prompt Tokens:{cb.prompt_tokens}")
print(f"Completion Tokens:{cb.completion_tokens}")
print(f"Total Cost:{cb.total_cost}")

Total Tokens:1454
Prompt Tokens:1089
Completion Tokens:365
Total Cost:0.0023635


In [47]:
quiz = response.get('quiz')
quiz = json.loads(quiz)

In [48]:
quiz

{'1': {'mcq': 'What is the main goal of machine learning?',
  'options': {'a': 'Achieving artificial intelligence',
   'b': 'Solving practical problems',
   'c': 'Using symbolic approaches',
   'd': 'Borrowing methods from statistics'},
  'correct': 'Solving practical problems'},
 '2': {'mcq': 'Which subfield of AI did machine learning grow out of?',
  'options': {'a': 'Probabilistic reasoning',
   'b': 'Expert systems',
   'c': 'Neural networks',
   'd': 'Symbolic methods'},
  'correct': 'Neural networks'},
 '3': {'mcq': 'What caused a rift between AI and machine learning?',
  'options': {'a': 'Data acquisition problems',
   'b': 'Theoretical problems',
   'c': 'Knowledge-based approach',
   'd': 'Practical problems'},
  'correct': 'Knowledge-based approach'},
 '4': {'mcq': 'Which field did machine learning shift its focus towards in the 1990s?',
  'options': {'a': 'Artificial intelligence',
   'b': 'Pattern recognition',
   'c': 'Computer science',
   'd': 'Fuzzy logic'},
  'correct'

In [49]:
quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

In [50]:
quiz_table_data

[{'MCQ': 'What is the main goal of machine learning?',
  'Choices': 'a: Achieving artificial intelligence | b: Solving practical problems | c: Using symbolic approaches | d: Borrowing methods from statistics',
  'Correct': 'Solving practical problems'},
 {'MCQ': 'Which subfield of AI did machine learning grow out of?',
  'Choices': 'a: Probabilistic reasoning | b: Expert systems | c: Neural networks | d: Symbolic methods',
  'Correct': 'Neural networks'},
 {'MCQ': 'What caused a rift between AI and machine learning?',
  'Choices': 'a: Data acquisition problems | b: Theoretical problems | c: Knowledge-based approach | d: Practical problems',
  'Correct': 'Knowledge-based approach'},
 {'MCQ': 'Which field did machine learning shift its focus towards in the 1990s?',
  'Choices': 'a: Artificial intelligence | b: Pattern recognition | c: Computer science | d: Fuzzy logic',
  'Correct': 'Pattern recognition'},
 {'MCQ': 'What method was reinvented in the mid-1980s by neural networks researche

In [52]:
quiz = pd.DataFrame(quiz_table_data)

quiz.to_csv('machinelearning.csv', index=False)